In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from sklearn.model_selection import train_test_split
import fiona
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

# Load data file
df = pd.read_csv('comed_201910.csv')

# Filter out only residential accounts
df_residential = df[df['service_name'].str.contains('residential', case=False, na=False)]

# Filter for one day, ex. 2019-10-01
desired_date = '2019-10-01'
df_residential = df_residential[df_residential['date_time'].str.contains(desired_date)]

# Convert the time from HH:MM to seconds past since the beginning of the day
df_residential['date_time'] = pd.to_datetime(df_residential['date_time'])
df_residential['seconds_past'] = df_residential['date_time'].dt.hour * 3600 + df_residential['date_time'].dt.minute * 60

# Aggregate data to find time of maximum energy consumption for each zip code
df_grouped = df_residential.groupby('zip5')['energy'].idxmax()
peak_times = df_residential.loc[df_grouped, ['zip5', 'energy', 'seconds_past']]
peak_times.columns = ['zip_code', 'peak_energy_value', 'peak_energy_time']

# Load the geojson file
gdf = gpd.read_file('Chicago_ZC.geojson')

# Convert the 'GEOID20' column to integer for join operation
gdf['GEOID20'] = gdf['GEOID20'].astype(int)

# Merge the peak_times DataFrame with the gdf GeoDataFrame based on 'zip_code'
gdf = gdf.merge(peak_times, left_on='GEOID20', right_on='zip_code', how='left')

# Convert the geometries to a projected CRS
gdf = gdf.to_crs('EPSG:3857')

# Calculate the centroid of each polygon
gdf['centroid'] = gdf['geometry'].centroid

# Convert back to geographic for lat/long coordinates
gdf = gdf.to_crs('EPSG:4269')

# Convert the centroid to latitude and longitude coordinates
gdf['Lat_centroid'] = gdf['centroid'].y
gdf['Long_centroid'] = gdf['centroid'].x

# Keep only the desired columns
gdf = gdf[['zip_code', 'peak_energy_value', 'peak_energy_time', 'Lat_centroid', 'Long_centroid']]

# Convert 'zip_code' to int, handling NaN values which cannot be converted to int
gdf['zip_code'] = gdf['zip_code'].astype('Int64')

gdf = gdf.dropna()

# Convert back to GeoDataFrame
gdf['geometry'] = gpd.points_from_xy(gdf.Long_centroid, gdf.Lat_centroid)
gdf = gpd.GeoDataFrame(gdf, geometry='geometry')
gdf.set_crs("EPSG:4269", inplace=True)

# Load the land use shapefile into a GeoDataFrame
gdb_folder = 'Landuse2018_CMAP_v1.gdb'

# Function to load the data in chunks, using only the necessary columns
def load_data(gdb_folder, layer):
    # Residential land use codes
    residential_codes = ['1000', '1100', '1110', '1111', '1112', '1130', '1140', '1150', '1151']
    
    with fiona.Env():
        with fiona.open(gdb_folder, layer=layer) as src:
            # Read all records
            records = [{'geometry': f['geometry'], 'LANDUSE': f['properties']['LANDUSE']} 
                       for f in src if f is not None and 'geometry' in f]
            
            df = pd.DataFrame.from_records(records)
            gdf = gpd.GeoDataFrame(df, geometry='geometry', crs=src.crs)

    # Filter the GeoDataFrame to include only desired land use categories
    gdf = gdf[gdf['LANDUSE'].isin(residential_codes)]
    
    # Now split into chunks
    gdfs = [gdf[i:i+chunksize] for i in range(0, gdf.shape[0], chunksize)]
    return gdfs

chunksize = 50000

land_use = load_data(gdb_folder, 'Landuse2018_Dissolved_v1')

print("yes0")

# Convert the land use data to the same CRS as the zip code data
land_use = [lu_gdf.to_crs(gdf.crs) for lu_gdf in land_use]

# Initialize an empty list to hold the results
land_use_joined = []

for lu_gdf in land_use:
    # Perform a spatial join to associate each land use parcel with its corresponding zip code
    joined = gpd.sjoin(lu_gdf, gdf[['geometry', 'zip_code']], how='left', op='within')
    land_use_joined.append(joined)

# Combine the results
land_use = pd.concat(land_use_joined)

land_use = land_use.to_crs('EPSG:3857')

# Calculate the area of each land use type in each zip code
land_use['area'] = land_use.geometry.area

land_use = land_use.to_crs(gdf.crs)

# Summarize land use area by land use type and zip code
land_use_summary = land_use.groupby(['zip_code', 'LANDUSE']).area.sum().unstack().reset_index()

# Fill NA values with 0 (no land of that type in the zip code)
land_use_summary.fillna(0, inplace=True)

# Calculate the total area of each zip code
total_area = land_use_summary.iloc[:, 1:].sum(axis=1)

# Normalize the land use areas by the total area of each zip code
for col in land_use_summary.columns[1:]:
    land_use_summary[col] = land_use_summary[col] / total_area

# Join the land use data with the original DataFrame
gdf = gdf.merge(land_use_summary, on='zip_code', how='left')

# Load the data
data = gdf

# Split the data into X and Y variables
# Include all columns that are not 'peak_energy_time' and 'geometry' as input features (including the new land use features)
X = data.drop(columns=['peak_energy_time', 'geometry']) 
Y = data['peak_energy_time']

# Split the data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# *** THIS CODE USES RANDOMIZED SEARCH AND IS LESS COMPUTATIONALLY EXPENSIVE THAN GRID SEARCH *** 

# Refining hyperparameters for XGBoost
xgb_param_grid = {
    'n_estimators': list(range(135, 145)),  # Focusing more on 140
    'max_depth': [6],  # Optimal value from the last search
    'learning_rate': np.linspace(0.045, 0.055, 20),  # Focusing more around 0.049
    'gamma': np.linspace(0.33, 0.37, 20),  # Focusing more around 0.35
    'colsample_bytree': [1.0],  # Optimal value from the last search
    'subsample': np.linspace(0.80, 0.85, 20)  # Focusing more around 0.82
}

xgb_model = xgb.XGBRegressor()

xgb_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=xgb_param_grid, n_iter=100, cv=5, random_state=42)
xgb_search.fit(X_train, Y_train)

print("Best XGBoost Parameters:", xgb_search.best_params_)
print("Best XGBoost Score:", xgb_search.best_score_)

# Refining hyperparameters for RandomForest
rf_param_grid = {
    'n_estimators': list(range(235, 245)),  # Focusing more around 241
    'max_depth': list(range(20, 23)),  # Focusing more around 21
    'min_samples_split': [8],  # Optimal value from the last search
    'min_samples_leaf': [1],  # Optimal value from the last search
    'bootstrap': [True]  # Optimal value from the last search
}

rf_model = RandomForestRegressor(random_state=42)

rf_search = RandomizedSearchCV(estimator=rf_model, param_distributions=rf_param_grid, n_iter=100, cv=5, random_state=42)
rf_search.fit(X_train, Y_train)

print("Best RandomForest Parameters:", rf_search.best_params_)
print("Best RandomForest Score:", rf_search.best_score_)

yes0


/Users/sarthak/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3400: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/Users/sarthak/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3400: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/Users/sarthak/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3400: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/Users/sarthak/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3400: FutureWarning: The `op` parameter is deprecated and will be removed

Best XGBoost Parameters: {'subsample': 0.8315789473684211, 'n_estimators': 144, 'max_depth': 6, 'learning_rate': 0.04657894736842105, 'gamma': 0.3342105263157895, 'colsample_bytree': 1.0}
Best XGBoost Score: 0.3374239468956142


/Users/sarthak/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 30 is smaller than n_iter=100. Running 30 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best RandomForest Parameters: {'n_estimators': 240, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_depth': 21, 'bootstrap': True}
Best RandomForest Score: 0.3622693455981518
